In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Compare generating from angles with generating from distances. 

In [ ]:
# plot from paper
fname = 'results/angles_distances_ICASSP.pkl'
plot_sigmas = [-5, -3, -1]

df_results = pd.read_pickle(fname)
df_results.head()
df_results.loc[:, 'square_size'] = df_results.loc[:, 'size']
df_results.loc[:, 'n_it'] = df_results.n_it.astype(np.float16)
print(df_results.square_size.values)

In [ ]:
from helpers import savefig
sigmas_distance = df_results[df_results.type=='distance'].sigma.unique()
print('available sigmas distance', sigmas_distance)
sigmas_angle = df_results[df_results.type=='angle'].sigma.unique()
print('available sigmas distance', sigmas_angle)

for sigma_distance in sigmas_distance[-4:]:
    data=df_results[(df_results.type=='angle') | ((df_results.type=='distance') & (df_results.sigma==sigma_distance))]

    sizes = data.square_size.unique()[::2]
    delta=4
    colors = sns.color_palette('Blues', n_colors=len(sizes)+delta)
    colors_dist = sns.color_palette('Reds', n_colors=len(sizes)+delta)

    fig, ax = plt.subplots()
    fig.set_size_inches(6, 2.5)
    i = 0
    l2s=[]
    for size in sizes:
        data_size = data[data.square_size==size]
        error_distance = data_size.loc[data_size.type=='distance', 'error'].mean()
        error_angle = data_size.loc[data_size.type=='angle']

        df = error_angle[['sigma','error']].astype(np.float32)
        df_means = df.groupby('sigma').agg('median')
        ax.loglog(df_means.index.values, df_means.error.values, color=colors[i+delta], label=size)
        l2 = ax.axhline(y=error_distance, color=colors_dist[i+delta])
        l2s.append(l2)
        i+=1
    ax.set_title('$\sigma_d$={:.1e}'.format(sigma_distance))
    ax.set_xlabel('$\sigma_a$')
    ax.set_ylabel('MSE')
    handels, labels = ax.get_legend_handles_labels()
    ax.legend(handels[:2]+handels[-2:], 
              labels[:2]+labels[-2:],
              loc='lower right', title='size of area')
    #savefig('plots/comparison.pdf', fig=fig)

In [ ]:
from helpers import savefig
sigmas_distance = df_results[df_results.type=='distance'].sigma.unique()
sigmas_angle = df_results[df_results.type=='angle'].sigma.unique()

fig, ax = plt.subplots()
fig.set_size_inches(6, 2.5)

sizes = data.square_size.unique()[::2]
delta=4
colors = sns.color_palette('Blues', n_colors=len(sizes)+delta)
max_sigma = 10

for i, size in enumerate(sizes):
    data_size = data[data.square_size==size]
    error_angle = data_size.loc[data_size.type=='angle']
    
    sigmas = df_means.index.values
    xs = sigmas[sigmas<=max_sigma]
    ys = df_means.error.values[sigmas<=max_sigma]

    df = error_angle[['sigma','error']].astype(np.float32)
    df_means = df.groupby('sigma').agg('median')
    ax.loglog(xs, ys, color=colors[i+delta], label=size)

palettes = ['Greens', 'Oranges', 'Purples']
linestyles = [':', '-.', '--']
chosen_sigmas = [sigmas_distance[i] for i in plot_sigmas]
handles2 = []
for j, sigma_distance in enumerate(chosen_sigmas):
    data=df_results[(df_results.type=='angle') | ((df_results.type=='distance') & (df_results.sigma==sigma_distance))]
    colors_dist = sns.color_palette(palettes[j], n_colors=len(sizes)+delta)
    for i, size in enumerate(sizes):
        data_size = data[data.square_size==size]
        error_distance = data_size.loc[data_size.type=='distance', 'error'].mean()
        ax.axhline(y=error_distance, color=colors_dist[i+delta], ls=linestyles[j])
    h2 = ax.axhline(y=error_distance, color=colors_dist[i+delta], ls=linestyles[j])
    handles2.append(h2)
    
ax.set_xlabel('angle noise $\sigma_a$')
ax.set_ylabel('accuracy')
ax.grid()
handels, labels = ax.get_legend_handles_labels()
l1 = ax.legend(handels[:1]+handels[-1:], 
          labels[:1]+labels[-1:],
          loc='lower right', title='size of area', framealpha=1.0)
ax.legend(handles2, ['{:2.1e}'.format(s) for s in chosen_sigmas], 
          loc='upper left', title='distance noise $\sigma_d$', framealpha=1.0)
ax.add_artist(l1)
savefig('plots/comparison.pdf', fig=fig)